In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

Data preprocessing

In [8]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
Y = dataset.iloc[:, -1].values

In [32]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [19]:
# Encode categorical variables

# Label encoding gender column
# note: one hot might be better
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])



In [36]:
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(sparse=False)
# ohe.fit_transform(le.fit_transform(X[:, 1]).reshape(-1, 1))




from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))




In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


Feature scaling

In [88]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Build the ANN

In [89]:

# ann = tf.keras.models.Sequential()

# # add input layer and 2 hidden layer
# ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
# ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
# ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(12,)),
    tf.keras.layers.Dense(units=6, activation='relu'),
    tf.keras.layers.Dense(units=6, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid'),
])



Train the ANN

In [90]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [91]:
ann.fit(X_train, Y_train, batch_size=128, epochs=100)

Epoch 1/100
63/63 [==============================] - 3s 5ms/step - loss: 0.7171 - accuracy: 0.4900
Epoch 2/100
63/63 [==============================] - 0s 4ms/step - loss: 0.6414 - accuracy: 0.7536
Epoch 3/100
63/63 [==============================] - 0s 5ms/step - loss: 0.5810 - accuracy: 0.7933
Epoch 4/100
63/63 [==============================] - 0s 5ms/step - loss: 0.5321 - accuracy: 0.7960
Epoch 5/100
63/63 [==============================] - 0s 6ms/step - loss: 0.4985 - accuracy: 0.7960
Epoch 6/100
63/63 [==============================] - 0s 6ms/step - loss: 0.4765 - accuracy: 0.7960
Epoch 7/100
63/63 [==============================] - 0s 5ms/step - loss: 0.4617 - accuracy: 0.7961
Epoch 8/100
63/63 [==============================] - 0s 4ms/step - loss: 0.4513 - accuracy: 0.7979
Epoch 9/100
63/63 [==============================] - 0s 5ms/step - loss: 0.4435 - accuracy: 0.7989
Epoch 10/100
63/63 [==============================] - 0s 5ms/step - loss: 0.4378 - accuracy: 0.8004
Epoch 11/

In [92]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
ann.predict([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])

1/1 [==============================] - 0s 266ms/step
[[0.44875535]]
1/1 [==============================] - 0s 336ms/step


array([[0.44875535]], dtype=float32)

In [96]:
print(ann.predict([[0, 0, 1,608,0,41,1,83807.86,1,0,1,112542.58]]))
ann.predict(sc.transform([[0, 0, 1,608,0,41,1,83807.86,1,0,1,112542.58]]))

1/1 [==============================] - 0s 60ms/step
[[0.44875535]]
1/1 [==============================] - 0s 33ms/step


array([[0.44875535]], dtype=float32)

In [97]:
ann.predict(sc.transform([[0, 0, 1,608,0,41,1,83807.86,1,0,1,112542.58]])) > 0.5

1/1 [==============================] - 0s 22ms/step


array([[False]])

Confusion matrix

In [105]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(-1, 1), Y_test.reshape(-1, 1)), 1))


63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [106]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_pred=y_pred, y_true=Y_test)
print(cm)
print(accuracy_score(Y_test, y_pred))


[[1506   89]
 [ 197  208]]
0.857
